## I. Import thư viện

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import csv
import time
import random
from tqdm import tqdm
import pandas as pd

## II. Lấy các URL từ Chrome

In [17]:
countries = ['vietnam', 'thailand', 'singapore', 'malaysia', 'indonesia', 'cambodia', 'philippines']

countries_links = ['https://www.traveloka.com/vi-vn/hotel/' + country for country in countries]
regions_links = []
hotels_links = []

### 1. Crawl link khách sạn theo quốc gia

In [ ]:
driver = webdriver.Chrome()

for country in countries:
    country_link = f'https://www.traveloka.com/vi-vn/hotel/{country}'
    driver.get(country_link)
    time.sleep(5)

    soup_country = BeautifulSoup(driver.page_source, 'html.parser')  # Lấy source HTML

    for link in soup_country.find_all('a', class_='css-4rbku5', href=True):
        href = link['href']
        if href.startswith(f'/vi-vn/hotel/{country}'):
            full_url = 'https://www.traveloka.com' + href

            suffix = href[len(f'/vi-vn/hotel/{country}/'):]  # phần sau /vi-vn/hotel/{country}/
            first_segment = suffix.split('/')[0]

            if first_segment in ['region', 'area', 'city', 'landmark']:
                if full_url not in regions_links:
                    regions_links.append(full_url)
            else:
                if full_url not in hotels_links:
                    hotels_links.append(full_url)      

print(f'Tìm thấy {len(regions_links)} vùng khách sạn và {len(hotels_links)} khách sạn.')


In [ ]:
# regions_links.clear()
# hotels_links.clear()

### 2. Crawl link khách sạn theo phân vùng của mỗi quốc gia

In [ ]:
# b. Crawl phân vùng những lần sau (chạy trước khi chạy cell dưới)

# append data trong file regions_links.csv vào list regions_links
with open('regions_links.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)  # Bỏ dòng tiêu đề
    for row in reader:
        if row:
            regions_links.append(row[0])

# URL bắt đầu và kết thúc
start_url = "https://www.traveloka.com/vi-vn/hotel/malaysia/area/dengkil-108048"
end_url   = "https://www.traveloka.com/vi-vn/hotel/philippines/city/lubao-10008325"

# Xác định vị trí index bắt đầu và kết thúc
try:
    start_index = regions_links.index(start_url)
    end_index = regions_links.index(end_url)
except ValueError as e:
    print("Không tìm thấy URL trong danh sách:", e)
    exit()

# Giới hạn danh sách cần crawl
regions_links_to_crawl = regions_links[start_index:end_index + 1]

In [ ]:
# a. Crawl phân vùng (đổi thành in regions_links_to_crawl ở những lần crawl sau)

driver = webdriver.Chrome()

for region_link in regions_links_to_crawl: # lần đầu thì in regions_links
    driver.get(region_link)
    time.sleep(random.uniform(3,7))

    soup_region = BeautifulSoup(driver.page_source, 'html.parser')  # Lấy source HTML

    for link in soup_region.find_all('a', class_='css-4rbku5', href=True):
        href = link['href']
        if href.startswith(f'/vi-vn/hotel/'):
            full_url = 'https://www.traveloka.com' + href

            suffix = href[len(f'/vi-vn/hotel/'):]  # phần sau /vi-vn/hotel/{country}/
            segments = suffix.split('/')

            if len(segments) >=2:
                if segments[1] in ['region', 'area', 'city', 'landmark']:
                    if full_url not in regions_links:
                        regions_links.append(full_url)
                else:
                    if full_url not in hotels_links:
                        hotels_links.append(full_url)

In [ ]:
print('\n'.join(regions_links))

In [ ]:
# Ghi regions_links ra file CSV (sau mỗi lần crawl phân vùng khách sạn thì chạy lại)
with open('regions_links.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Region URL'])  # Tiêu đề cột
    for link in regions_links:
        writer.writerow([link])  # Mỗi link là một dòng

In [ ]:
# Ghi hotels_links ra file CSV (chỉ chạy khi crawl phân vùng lần đầu)
with open('hotels_links.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Hotel URL'])  # Tiêu đề cột
    for link in hotels_links:
        writer.writerow([link])  # Mỗi link là một dòng

In [ ]:
# Ghi tiếp các URL mới vào hotels_links.csv (chạy khi crawl phân vùng những lần sau)
with open('hotels_links.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    for url in hotels_links:
        writer.writerow([url])

In [ ]:
print('\n'.join(hotels_links))

### 3. Crawl link của từng khách sạn ở Việt Nam

In [ ]:
# crawl khách sạn những lần sau (chạy trước khi chạy cell dưới)

regions_links = []

# append data trong file regions_links.csv vào list regions_links
with open('regions_links.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)  # Bỏ dòng tiêu đề
    for row in reader:
        if row:
            regions_links.append(row[0])

# URL bắt đầu và kết thúc (chỉ lấy của vietnam)
start_url = "https://www.traveloka.com/vi-vn/hotel/vietnam/city/bao-loc-city-10011056"  # dòng thứ 5198 file regions_links.csv
end_url   = "https://www.traveloka.com/vi-vn/hotel/vietnam/area/ward-14-30010903/2" # dòng thứ 5500 file regions_links.csv

# Xác định vị trí index bắt đầu và kết thúc
try:
    start_index = regions_links.index(start_url)
    end_index = regions_links.index(end_url)
except ValueError as e:
    print("Không tìm thấy URL trong danh sách:", e)
    exit()

# Giới hạn danh sách cần crawl
regions_links_to_crawl = regions_links[start_index:end_index + 1]

In [ ]:
# crawl khách sạn (đổi thành in regions_links_to_crawl ở những lần crawl sau)

vietnam_hotels_links = []

driver = webdriver.Chrome()

for region_link in regions_links_to_crawl: # lần đầu thì in regions_links
    driver.get(region_link)
    time.sleep(random.uniform(3,7))

    soup = BeautifulSoup(driver.page_source, 'html.parser')  # Lấy source HTML

    for link in soup.find_all('a', class_='css-4rbku5 r-13awgt0', href=True):
        href = link['href']
        if href.startswith(f'/vi-vn/hotel/'):
            full_url = 'https://www.traveloka.com' + href

        if full_url not in vietnam_hotels_links:
            vietnam_hotels_links.append(full_url)

In [ ]:
print('\n'.join(vietnam_hotels_links))

In [ ]:
# Ghi vietnam_hotels_links ra file CSV (chỉ chạy khi crawl khách sạn lần đầu)
with open('vietnam_hotels_links.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Hotel URL'])  # Tiêu đề cột
    for link in vietnam_hotels_links:
        writer.writerow([link])  # Mỗi link là một dòng

In [ ]:
# Ghi tiếp các URL mới vào vietnam_hotels_links.csv (chạy khi crawl khách sạn những lần sau)
with open('vietnam_hotels_links.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    for url in vietnam_hotels_links:
        writer.writerow([url])

## III. Crawl các thuộc tính của khách sạn Việt Nam

### 1. Gom các links khách sạn ở file hotels_links.csv và file vietnam_hotels_links.csv lại

In [ ]:
all_links = []

# append data trong file hotels_links.csv vào list all_links
with open('hotels_links.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)  # Bỏ dòng tiêu đề
    for row in reader:
        if row:
            all_links.append(row[0])

# lọc link khách sạn Việt Nam
vietnam_links = [link for link in all_links if link.startswith("https://www.traveloka.com/vi-vn/hotel/vietnam")]

In [ ]:
print('\n'.join(vietnam_links))
# URL khách sạn vietnam trong file hotels_links.csv (file chung, được append trong quá trình crawl phân vùng)

In [ ]:
vietnam_hotels_links = []

# append data trong file vietnam_hotels_links.csv vào list vietnam_hotels_links
with open('vietnam_hotels_links.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)  # Bỏ dòng tiêu đề
    for row in reader:
        if row:
            vietnam_hotels_links.append(row[0])

In [ ]:
print('\n'.join(vietnam_hotels_links))
# URL khách sạn trong file vietnam_hotels_links.csv (được crawl ở II.3.)

In [ ]:
# Chuyển vietnam_hotels_links thành set để kiểm tra nhanh
existing = set(vietnam_hotels_links)

# Thêm từng phần tử trong vietnam_links nếu chưa có trong vietnam_hotels_links
for item in vietnam_links:
    if item not in existing:
        vietnam_hotels_links.append(item)
        existing.add(item)  # cập nhật vào set để tránh kiểm tra trùng lại

### 2. Crawl các thuộc tính

In [ ]:
vietnam_hotels_links = []

# append data trong file vietnam_hotels_links.csv vào list vietnam_hotels_links
with open('vietnam_hotels_links.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)  # Bỏ dòng tiêu đề
    for row in reader:
        if row:
            vietnam_hotels_links.append(row[0])

# URL bắt đầu và kết thúc (chỉ lấy của vietnam)
start_url = "https://www.traveloka.com/vi-vn/hotel/vietnam/sun-villa-tam-dao-9000001072334"  # dòng thứ x file vietnam_hotels_links.csv
end_url   = "https://www.traveloka.com/vi-vn/hotel/vietnam/bavet-yongyuan-casino--hotel-9000005623932" # dòng thứ y file vietnam_hotels_links.csv

# Xác định vị trí index bắt đầu và kết thúc
try:
    start_index = vietnam_hotels_links.index(start_url)
    end_index = vietnam_hotels_links.index(end_url)
except ValueError as e:
    print("Không tìm thấy URL trong danh sách:", e)
    exit()

# Giới hạn danh sách cần crawl
vn_hotels_links_to_crawl = vietnam_hotels_links[start_index:end_index + 1]

In [ ]:
print('\n'.join(vn_hotels_links_to_crawl)) # URL khách sạn trong file vietnam_hotels_links.csv

In [ ]:
def crawl_hotels_vn(url):
    options = Options()
    options.add_argument("--headless")  # Chạy ẩn trình duyệt
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(random.uniform(3,7))
    soup_hotel_vn = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

# url khách sạn
    hotel_url = url
    
# tên khách sạn
    hotel_name = soup_hotel_vn.find('h1', {'data-testid': 'display_name_label'}).text

# giá chung  data-testid="overview_cheapest_price"
    overview_price_div = soup_hotel_vn.find('div', {'data-testid': 'overview_cheapest_price'})  # Dùng find() thay vì find_all()
    if overview_price_div:
        overview_price = overview_price_div.text
    else:
        overview_price = None

# địa chỉ
    address_wrapper = soup_hotel_vn.find('div', {'data-testid': 'summary-location'})
    address = address_wrapper.find('div', class_='css-901oao css-cens5h r-13awgt0 r-uh8wd5 r-1b43r93 r-majxgm r-rjixqe r-fdjqy7').text

# điểm đánh giá chung
    overall_rating_wrapper = soup_hotel_vn.find('h2', {'data-testid': 'review-data-title'})
    overall_rating = overall_rating_wrapper.find_next('div', class_='css-901oao r-1i6uqv8 r-uh8wd5 r-s67bdx r-b88u0q r-1ptu69t r-q4m81j')
    if overall_rating:
        overall_rating = overall_rating.text

# điểm đánh giá 5 khía cạnh (vệ sinh, tiện nghi phòng, đồ ăn, vị trí, dịch vụ và tiện ích)
    rating_blocks = soup_hotel_vn.find_all('div', class_='css-1dbjc4n r-29ag51')
    texts = []
    if rating_blocks:
        for block in rating_blocks:
            texts.append(block.find('h3').text)
    else: texts = [None] *5

    cleanliness, room_comfort, meal, location, service_facilites = texts[:5]

# tiện ích chính & khoảng cách đến trung tâm thành phố & thời gian nhận/trả phòng
    section = soup_hotel_vn.find('div', {'data-testid': 'section-policy'})
    rows = section.find_all('tr')
    main_facilities = checkin_checkout_time = distance_to_downtown = None
    for row in rows:
        tds = row.find_all('td')
        label = tds[0].get_text(strip=True)
        if label == "Tiện ích chung":
            main_facilities = tds[1].get_text(strip=True)
        elif label == "Thời gian nhận/trả phòng":
            checkin_checkout_time = tds[1].get_text(strip=True)
        elif label == "Khoảng cách đến trung tâm thành phố":
            distance_to_downtown = tds[1].get_text(strip=True)

    return [hotel_url, hotel_name, overview_price, address, overall_rating, cleanliness, room_comfort, meal, location, service_facilites, main_facilities, checkin_checkout_time, distance_to_downtown]


In [ ]:
hotel_url, hotel_name, overview_price, address, overall_rating, cleanliness, room_comfort  = [], [], [], [], [], [], []
meal, location, service_facilites, main_facilities, checkin_checkout_time, distance_to_downtown = [], [], [], [], [], []

for url in tqdm(vn_hotels_links_to_crawl):
    try:
        hotels_vn_data = crawl_hotels_vn(url)
        hotel_url.append(url)
        hotel_name.append(hotels_vn_data[1])
        overview_price.append(hotels_vn_data[2])
        address.append(hotels_vn_data[3])
        overall_rating.append(hotels_vn_data[4])
        cleanliness.append(hotels_vn_data[5])
        room_comfort.append(hotels_vn_data[6])
        meal.append(hotels_vn_data[7])
        location.append(hotels_vn_data[8])
        service_facilites.append(hotels_vn_data[9])
        main_facilities.append(hotels_vn_data[10])
        checkin_checkout_time.append(hotels_vn_data[11])
        distance_to_downtown.append(hotels_vn_data[12])
    except Exception as e:
        print(f"Lỗi ở {url}: {e}")


In [ ]:
dataset_vn_hotels = pd.DataFrame({'Hotel URL': hotel_url,
                                  'Hotel Name': hotel_name,
                                  'Overview Price': overview_price,
                                  'Address': address,
                                  'Overall Rating': overall_rating,
                                  'Cleanliness': cleanliness,
                                  'Room Comfort':  room_comfort,
                                  'Meal':  meal,
                                  'Location':  location,
                                  'Service Facilities': service_facilites,
                                  'Main Facilities': main_facilities,
                                  'Checkin-Checkout Time': checkin_checkout_time,
                                  'Distance to Downtown': distance_to_downtown
                                  })
dataset_vn_hotels

,Hotel URL,Hotel Name,Overview Price,Address,Overall Rating,Cleanliness,Room Comfort,Meal,Location,Service Facilities,Main Facilities,Checkin-Checkout Time,Distance to Downtown
0,https://www.traveloka.com/vi-vn/hotel/vietnam/...,Sun Villa Tam Dao,None,"Sun Villa, Khu 2, Thị trấn Tam Đảo , Tam Dao T...","7,7","7,2","7,4","6,8","7,7","7,2","Máy lạnh, Nhà hàng, Hồ bơi, Lễ tân 24h, Chỗ đậ...",Từ 14:00 - Trước 12:00,419 m
1,https://www.traveloka.com/vi-vn/hotel/vietnam/...,Ngan Ha Hotel Tam Dao,None,"Khu 1, Tam Dao Township, Huyện Tam Đảo, Tỉnh ...","8,4","8,5","8,4","7,8","7,9","8,1","Máy lạnh, Nhà hàng, Lễ tân 24h, Chỗ đậu xe, Th...",Từ 14:00 - Trước 12:00,311 m


In [ ]:
print(hotels_vn_data) # URL khách sạn trong file vietnam_hotels_links.csv

['https://www.traveloka.com/vi-vn/hotel/vietnam/ngan-ha-hotel-tam-dao-3000020014093', 'Ngan Ha Hotel Tam Dao', None, 'Khu 1, Tam Dao Township, Huyện Tam Đảo, Tỉnh Vĩnh Phúc, Việt Nam, 280000', '8,4', '8,5', '8,4', '7,8', '7,9', '8,1', 'Máy lạnh, Nhà hàng, Lễ tân 24h, Chỗ đậu xe, Thang máy, WiFi', 'Từ 14:00 - Trước 12:00', '311 m']


Không thể lấy được thuộc tính overview_price của các khách sạn Traveloka vì phải giải capcha cho mỗi lần load trang \
Nhóm chuyển hướng chọn crawl các khách sạn ở trang Booking.com